In [ ]:
# This script takes in an [x] FPS video and outputs 2*[x] FPS video by applying results
#  of frame interpolation using FI-CNN.
import sys
import os

import warnings
warnings.filterwarnings("ignore")

BACKEND = "tensorflow"
os.environ['KERAS_BACKEND'] = BACKEND
os.environ['THEANO_FLAGS'] = "device=gpu0, lib.cnmem=0.85, optimizer=fast_run"

import time
import random

from keras.layers import Layer, Input, Dense,Convolution2D, MaxPooling2D, UpSampling2D, merge, BatchNormalization, Deconvolution2D, Activation
from keras.layers.advanced_activations import PReLU
from keras.models import Sequential, Model, clone_model

In [ ]:
def block_oneb(inp, cd, chanel='channels_first'):
    a = Convolution2D(cd, (3, 3), data_format=chanel, activation='relu', padding='same')(inp)
    #print("a",a.shape)
    b = BatchNormalization()(a)
    c = Convolution2D(cd, (3, 3), data_format=chanel, activation='relu', padding='same')(b)
    d = BatchNormalization()(c)
    #print("d", d.shape)
    e = MaxPooling2D(pool_size=(2, 2), data_format=chanel)(d)
    #print("e",e.shape)
    return d,e

def block_twob(ups_input,con_input,cd, pool=True, chanel='channels_first'):
    #print("before", ups_input.shape)
    ups = UpSampling2D(size=(2, 2), data_format=chanel,)(ups_input)
    #print("UPsampled", ups.shape)
    #print("to concat with:", con_input.shape)
    a = concatenate([ups, con_input], axis=-3)
    #print("after concat",a.shape)
    b = Convolution2D(cd, (3, 3), data_format=chanel, activation='relu', padding='same')(a)
    c = BatchNormalization()(b)
    d = Convolution2D(cd, (3, 3), data_format=chanel, activation='relu', padding='same')(c)
    e = BatchNormalization()(d)
    return e
    

In [ ]:
#from FI_unet import get_unet_2
from keras.layers.merge import concatenate
def get_unet_2(a):
    #1.
    inputs = Input(a)
    #print(a,inputs.shape)
    bn1,bl1 = block_oneb(inputs,32)
    #print("\tbl1",bl1.shape)
    #2
    bn2,bl2 = block_oneb(bl1,64)
    #print("\tbl2",bl2.shape)
    #3
    bn3,bl3 = block_oneb(bl2, 128)
    #print("\tbl3",bl3.shape)
    #4
    bn4,bl4 = block_oneb(bl3, 256)
    #print("\tbl4",bl4.shape)
    #print("\tbn4",bn4.shape)
    
    #5
    bn5,pool5 = block_oneb(bl4,512)
    #print("\tbl5",pool5.shape)
    
    #5_2
    bn5_2, pool5_2 = block_oneb(pool5,512)
    #print("bn5 -  no pool", bn5.shape)
    #print("bn5_2 - pool+conv", bn5_2.shape) # too big
    
    #6_2
    bn6_2 = block_twob(bn5_2, bn5, 512)
    #print("result", bn6_2.shape)

    #6
    bn6 = block_twob(bn6_2,bn4, 256)

    #7
    bn7 = block_twob(bn6,bn3, 128)

    #8
    bn8 = block_twob(bn7,bn2, 64)

    #9
    bn9 = block_twob(bn8,bn1, 32)

    #10 input_shape[0]/2
    conv10 = Convolution2D(3, (1, 1),data_format='channels_first', activation='sigmoid')(bn9)

    #print(conv10.shape)
    model = Model(input=inputs, outputs=conv10)
    #print(model.summary())

    return model

In [ ]:
shps = (6,128, 384)
model = get_unet_2(shps)